In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM, Flatten, Conv1D, LocallyConnected1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from math import sqrt
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from matplotlib import pyplot
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
import numpy as np
import argparse
import os
from keras.callbacks import CSVLogger
import keras.backend as K
import models

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## DATA PREP

In [6]:
import data_preparation
#Convert letters to integers
integer_encoded, params = data_preparation.process_data('input.txt')

15072434
{'A': 0, 'T': 1, 'G': 2, 'C': 3}
{0: 'A', 1: 'T', 2: 'G', 3: 'C'}
[[2]
 [3]
 [3]
 [1]
 [0]
 [0]
 [2]
 [3]
 [3]
 [1]]
GCCTAAGCCT


In [7]:
bs=128
time_steps=64
num_epochs=20

In [8]:
#Generate traning data and its lable 
X,Y = data_preparation.generate_single_output_data(integer_encoded[: 150722],bs, time_steps)

(150722, 1)
(150656, 65)


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Trainning

In [2]:
def custom_loss(y_true, y_pred):
        return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)        

In [ ]:
alphabet_size = Y.shape[1]
DZ_model = Sequential()
DZ_model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(Flatten())
DZ_model.add(Dense(64, activation='relu'))
DZ_model.add(Dense(alphabet_size, activation='softmax'))
optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
DZ_model.compile(loss=custom_loss, optimizer=optim)
early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
callbacks_list = [early_stopping]
DZ_model.fit(X, Y, epochs=num_epochs, batch_size=bs, verbose=1, shuffle=True, callbacks=callbacks_list)

Epoch 1/20
150656/150656 [==============================] - 150s 997us/step - loss: 1.8998
Epoch 2/20
150656/150656 [==============================] - 141s 933us/step - loss: 1.8690
Epoch 3/20
150656/150656 [==============================] - 139s 920us/step - loss: 1.8596
Epoch 4/20
 46080/150656 [========>.....................] - ETA: 1:36 - loss: 1.8457

## Compression

In [11]:
num_iters = int((len(X)+time_steps)/bs)
ind = np.array(range(bs))*num_iters

DZ_model = load_model('my_model.h5', compile=False)
prob = DZ_model.predict(X[ind,:], batch_size=bs)

In [18]:
print(X)
print(X[ind,:].shape)

[[2 3 3 ... 3 3 1]
 [3 3 1 ... 3 1 0]
 [3 1 0 ... 1 0 0]
 ...
 [3 2 2 ... 1 1 1]
 [2 2 3 ... 1 1 3]
 [2 3 2 ... 1 3 1]]
(128, 64)


In [19]:

print(prob.shape)

(128, 4)


In [22]:
print(sum(prob[1,:]))

1.0000000074505806
